In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

driver = webdriver.Chrome()
driver.maximize_window()


def load_page(url):
    driver.get(url)
    time.sleep(3)  # Wait for the page to load

# Function to scrape bus routes
def scrape_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name , state_name):
    try:
        driver.get(url)
        time.sleep(3)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(3)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "State_Name": state_name,
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages(state_name):
    for page in range(1, 6):  # There are 5 pages
        try:
            load_page(URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name, state_name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Define the state name for the scraping process
state_name = 'Uttar Pradesh'

# Scrape routes and details from all pages
scrape_all_pages(state_name)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('uttarpradesh.csv', index=False)
print("file was created successfully")
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/aligarh-uttar-pradesh-to-delhi: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A076FB05+28789]
	(No symbol) [0x00007FF7A06D86E0]
	(No symbol) [0x00007FF7A057592A]
	(No symbol) [0x00007FF7A05C930E]
	(No symbol) [0x00007FF7A05C95FC]
	(No symbol) [0x00007FF7A06128A7]
	(No symbol) [0x00007FF7A05EF47F]
	(No symbol) [0x00007FF7A060F654]
	(No symbol) [0x00007FF7A05EF1E3]
	(No symbol) [0x00007FF7A05BA938]
	(No symbol) [0x00007FF7A05BBAA1]
	GetHandleVerifier [0x00007FF7A0AA933D+3410093]
	GetHandleVerifier [0x00007FF7A0ABE7DD+3497293]
	GetHandleVerifier [0x00007FF7A0AB2A73+3448803]
	GetHandleVerifier [0x00007FF7A0837BBB+848171]
	(No symbol) [0x00007FF7A06E3C3F]
	(No symbol) [0x00007FF7A06DF6E4]
	(No symbol) [0x00007FF7A06DF87D]
	(No symbol) [0x00007FF7A06CED49]
	BaseThreadInitThunk [0x00007FFD3869259D+29]
	RtlUserThreadStart [0x00007FFD3A4EAF38+40]

Error occurred while scraping bus details for https://www.redbu

In [11]:
data = pd.read_csv('uttarpradesh.csv')
data.head()

,State_Name,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Uttar Pradesh,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - KSB0086,Volvo AC Seater 2+2,17:01,05h 30m,22:31,3.0,INR 894,20 Seats available
1,Uttar Pradesh,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - BLY0111,Ordinary Non AC Seater 2+3,19:01,04h 44m,23:45,4.9,INR 648,30 Seats available
2,Uttar Pradesh,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - GRH0203,Ordinary Non AC Seater 2+3,19:30,06h 30m,02:00,3.3,INR 493,52 Seats available
3,Uttar Pradesh,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - HDD0063,Ordinary Non AC Seater 2+3,19:39,05h 24m,01:03,3.3,INR 456,52 Seats available
4,Uttar Pradesh,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0128,Ordinary Non AC Seater 2+3,20:01,05h 00m,01:01,3.3,INR 648,36 Seats available


In [13]:
data.shape

(1061, 11)